In [258]:
import pandas as pd
import numpy as np

In [259]:
import os
this_directory = os.path.realpath(".")
home_directory = os.path.split(this_directory)[0]
data_directory = os.path.join(home_directory, "data")

In [260]:
## !!! make sure that the newer version isn't a pickle
try : 
    all_tracts = pd.read_pickle(f"{data_directory}/acs5-15.pkl").drop(columns=['Unnamed: 0'])
except : 
    all_tracts = pd.read_csv(f"{data_directory}/acs5-15.csv").drop(columns=['Unnamed: 0'])


qoz_numbers = pd.read_pickle(f"{data_directory}/qozs_clean.pkl")['census_tract_number']
all_tracts['tract_number'] = qoz_numbers
all_tracts['state'] = all_tracts['tract_number'].apply(lambda row : row[:2])

In [261]:
## look at lines where the fetcher didn't find anything
## approximately 1%
nothing = all_tracts[all_tracts['population_total'] == -1]
states_nothing = nothing.groupby('state').agg({'state':'count'})

from us import states
states_nothing['name'] = [ states.lookup(row).name for row in states_nothing.index.values]

* Census API didn't return anything for American Samoa, Guam, NM and Virgin Islands
* Could either impute another tract each for AZ and SD, (or just leave them out)

In [262]:
nothing_indexes = nothing.index.values
nothing_indexes

array([ 183,  184,  185,  186,  187,  188,  189,  190,  191,  192,  193,
        194,  195,  196,  197,  198,  341, 2266, 2267, 2268, 2269, 2270,
       2271, 2272, 2273, 2274, 2275, 2276, 2277, 2278, 2279, 2280, 2281,
       2282, 2283, 2284, 2285, 2286, 2287, 2288, 2289, 2290, 5433, 5434,
       5435, 5436, 5437, 5438, 5439, 5440, 5441, 5442, 5443, 5444, 5445,
       5446, 5447, 5448, 5449, 5450, 5451, 5452, 7319, 8199, 8200, 8201,
       8202, 8203, 8204, 8205, 8206, 8207, 8208, 8209, 8210, 8211, 8212])

In [263]:
## Check out if some of the QOZ lines where nothing was found were taken from acs 2016
QOZs = pd.read_pickle(f"{data_directory}/qozs_clean.pkl")
QOZs.groupby('acs_data_source').agg({'acs_data_source':'count'})
len(QOZs[QOZs['acs_data_source'].isna()])

0

In [264]:
acs2016 = QOZs[QOZs['acs_data_source'] == '2012-2016']
acs2016_indexes = acs2016.index.values

print(nothing.tract_number.values[np.isin(nothing_indexes, acs2016_indexes)])

## Nope, that wasn't the problem

[]


In [265]:
## get rid of lines where the fetcher didn't find anything at all

# 1 : tracts where the api returned nothing
tracts = all_tracts[all_tracts['population_total'] != -1]

# 2 : those 2 tracts from AZ and SD
tracts = tracts.drop(index=[1852, 3862])

tracts.info()

## We have nan's in mobility, home_value, and household_income

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8685 entries, 0 to 8763
Data columns (total 25 columns):
population_total            8685 non-null float64
marriage_total              8685 non-null float64
marriage_never_married      8685 non-null float64
mobility_total              7822 non-null float64
mobility_same_house_1yr     7822 non-null float64
mobility_same_county_1yr    7822 non-null float64
age_median                  8685 non-null float64
race_total                  8685 non-null float64
race_white                  8685 non-null float64
race_black                  8685 non-null float64
poverty_total               8685 non-null float64
poverty_poor                8685 non-null float64
household_income_median     8673 non-null float64
tenure_total                8685 non-null float64
tenure_renters              8685 non-null float64
occupancy_total             8685 non-null float64
occupancy_vacant            8685 non-null float64
home_value_median           8570 non-null f

In [266]:
nan_lines = tracts[tracts.isna().any(axis=1)]
nan_lines['state'] = nan_lines['tract_number'].apply(lambda row : row[:2])
nan_lines.groupby('state').agg({'state':'count'})
## 10% of opportunity zones are in Puerto Rico, which make up 90% of their tracts

# tracts.groupby('state').agg({'state':'count'})

,state
state,
01,1
02,1
04,3
06,19
08,2
09,1
11,1
12,5
13,5


* Previous residences were not recorded for tracts in Puerto Rico (72)
* Missing median home value and household income values were all across different states

In [267]:
## mobility
pd.options.display.max_rows = 100
mobility_nan = tracts[tracts['mobility_total'].isna()]
mobility_nan.groupby('state').agg({'state':'count'})

## TO DO: decide what to do with mobility for Puerto Rico

,state
state,
72,863


In [268]:
home_value_nan = tracts[tracts['home_value_median'].isna()]
#home_value_nan.groupby('state').agg({'state':'count'})

In [269]:
house_income_nan = tracts[tracts['household_income_median'].isna()]
#house_income_nan.groupby('state').agg({'state':'count'})

In [270]:
non_nans = tracts[tracts.notna().all(axis=1)] ## 7702 w/o nans, 985 w/nans
print(len(tracts), 'should equal', len(non_nans) + len(nan_lines)) 

8685 should equal 8685


### Cleaning

In [271]:
states , tract_numbers = tracts['state'], tracts['tract_number']

In [272]:
tracts.head()
## replace negative values (not found) with nan
tracts = tracts.drop(columns=['state','tract_number'])
tracts.where(tracts >= 0, inplace=True, errors='ignore') ## ignore comparisons of str and int
tracts.iloc[6945]

population_total            15.0
marriage_total              15.0
marriage_never_married      15.0
mobility_total               NaN
mobility_same_house_1yr      NaN
mobility_same_county_1yr     NaN
age_median                   NaN
race_total                  15.0
race_white                  15.0
race_black                   0.0
poverty_total               15.0
poverty_poor                15.0
household_income_median      NaN
tenure_total                15.0
tenure_renters              15.0
occupancy_total             19.0
occupancy_vacant            12.0
home_value_median            NaN
structure_year_median        NaN
structure_units_total       19.0
structure_units_one_a        0.0
structure_units_one_d       19.0
structure_units_mobile       0.0
Name: 7009, dtype: float64

In [273]:
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 40
pop = tracts['population_total']
# note: race_total, poverty_total = population_total
# mobility_total = total population > 1 year
#     going to just use total population, looks closer than tenure_total
# tenure_total = total population in occupied housing units
# poverty = universe of people for whom poverty status is determined (!)

tracts['p_never_married'] = round(tracts['marriage_never_married']/tracts['marriage_total'], 3)
tracts['p_outofcountyflux'] = round((tracts['mobility_same_house_1yr'] + tracts['mobility_same_county_1yr']) / pop, 3)
tracts['p_black'] = round(tracts['race_black'] / pop, 3)
tracts['p_white'] = round(tracts['race_white'] / pop, 3)
tracts['p_poverty'] = round(tracts['poverty_poor'] / pop, 3)
tracts['p_pop_renting'] = round(tracts['tenure_renters'] / tracts['tenure_total'], 3)
tracts.head()

,population_total,marriage_total,marriage_never_married,mobility_total,mobility_same_house_1yr,mobility_same_county_1yr,age_median,race_total,race_white,race_black,poverty_total,poverty_poor,household_income_median,tenure_total,tenure_renters,occupancy_total,occupancy_vacant,home_value_median,structure_year_median,structure_units_total,structure_units_one_a,structure_units_one_d,structure_units_mobile,p_never_married,p_outofcountyflux,p_black,p_white,p_poverty,p_pop_renting
0,2761.0,2112.0,532.0,406.0,2187.0,272.0,36.3,2761.0,2141.0,564.0,2761.0,797.0,34821.0,2715.0,949.0,1236.0,144.0,94100.0,1980.0,1236.0,591.0,8.0,578.0,0.252,0.891,0.204,0.775,0.289,0.350
1,2869.0,2248.0,447.0,485.0,2634.0,147.0,40.1,2869.0,2481.0,299.0,2869.0,826.0,31390.0,2861.0,574.0,1248.0,167.0,127600.0,1988.0,1248.0,827.0,7.0,306.0,0.199,0.969,0.104,0.865,0.288,0.201
2,4537.0,3709.0,872.0,785.0,4199.0,94.0,43.0,4537.0,4153.0,198.0,4513.0,676.0,44985.0,4537.0,700.0,2017.0,348.0,130300.0,1991.0,2017.0,1518.0,0.0,499.0,0.235,0.946,0.044,0.915,0.149,0.154
3,5321.0,4202.0,1072.0,857.0,4355.0,586.0,41.5,5321.0,4479.0,686.0,4747.0,954.0,41944.0,4755.0,1528.0,2219.0,327.0,131100.0,1975.0,2219.0,1807.0,31.0,23.0,0.255,0.929,0.129,0.842,0.179,0.321
4,3398.0,2429.0,1047.0,631.0,3038.0,297.0,29.3,3398.0,1273.0,2125.0,3377.0,748.0,27587.0,3379.0,1468.0,1550.0,287.0,92300.0,1978.0,1550.0,914.0,0.0,309.0,0.431,0.981,0.625,0.375,0.220,0.434


In [274]:
housing_units = tracts['structure_units_total']
## note : occupancy_total = structure_units_total
tracts['p_units_vacant'] = round(tracts['occupancy_vacant'] / housing_units, 3)
tracts['p_mobilehomes'] = round(tracts['structure_units_mobile'] / housing_units, 3)
tracts['p_multiple_unit_strucs'] = round((housing_units - (tracts['structure_units_one_a'] + tracts['structure_units_one_d'])) / housing_units, 3)
tracts.head()

,population_total,marriage_total,marriage_never_married,mobility_total,mobility_same_house_1yr,mobility_same_county_1yr,age_median,race_total,race_white,race_black,poverty_total,poverty_poor,household_income_median,tenure_total,tenure_renters,occupancy_total,occupancy_vacant,home_value_median,structure_year_median,structure_units_total,structure_units_one_a,structure_units_one_d,structure_units_mobile,p_never_married,p_outofcountyflux,p_black,p_white,p_poverty,p_pop_renting,p_units_vacant,p_mobilehomes,p_multiple_unit_strucs
0,2761.0,2112.0,532.0,406.0,2187.0,272.0,36.3,2761.0,2141.0,564.0,2761.0,797.0,34821.0,2715.0,949.0,1236.0,144.0,94100.0,1980.0,1236.0,591.0,8.0,578.0,0.252,0.891,0.204,0.775,0.289,0.350,0.117,0.468,0.515
1,2869.0,2248.0,447.0,485.0,2634.0,147.0,40.1,2869.0,2481.0,299.0,2869.0,826.0,31390.0,2861.0,574.0,1248.0,167.0,127600.0,1988.0,1248.0,827.0,7.0,306.0,0.199,0.969,0.104,0.865,0.288,0.201,0.134,0.245,0.332
2,4537.0,3709.0,872.0,785.0,4199.0,94.0,43.0,4537.0,4153.0,198.0,4513.0,676.0,44985.0,4537.0,700.0,2017.0,348.0,130300.0,1991.0,2017.0,1518.0,0.0,499.0,0.235,0.946,0.044,0.915,0.149,0.154,0.173,0.247,0.247
3,5321.0,4202.0,1072.0,857.0,4355.0,586.0,41.5,5321.0,4479.0,686.0,4747.0,954.0,41944.0,4755.0,1528.0,2219.0,327.0,131100.0,1975.0,2219.0,1807.0,31.0,23.0,0.255,0.929,0.129,0.842,0.179,0.321,0.147,0.010,0.172
4,3398.0,2429.0,1047.0,631.0,3038.0,297.0,29.3,3398.0,1273.0,2125.0,3377.0,748.0,27587.0,3379.0,1468.0,1550.0,287.0,92300.0,1978.0,1550.0,914.0,0.0,309.0,0.431,0.981,0.625,0.375,0.220,0.434,0.185,0.199,0.410


In [278]:
##
cleaned = tracts[['population_total', 'age_median', 'p_never_married', 'p_white', 'p_black',\
                  'p_poverty', 'household_income_median', 'home_value_median', 'structure_year_median', \
                  'p_outofcountyflux', 'p_pop_renting', 'p_units_vacant', 'p_mobilehomes', 'p_multiple_unit_strucs']]

In [279]:
cleaned['state'] = states
cleaned['tract'] = tract_numbers
cleaned.head()

,population_total,age_median,p_never_married,p_white,p_black,p_poverty,household_income_median,home_value_median,structure_year_median,p_outofcountyflux,p_pop_renting,p_units_vacant,p_mobilehomes,p_multiple_unit_strucs,state,tract
0,2761.0,36.3,0.252,0.775,0.204,0.289,34821.0,94100.0,1980.0,0.891,0.350,0.117,0.468,0.515,01,01001020700
1,2869.0,40.1,0.199,0.865,0.104,0.288,31390.0,127600.0,1988.0,0.969,0.201,0.134,0.245,0.332,01,01003010200
2,4537.0,43.0,0.235,0.915,0.044,0.149,44985.0,130300.0,1991.0,0.946,0.154,0.173,0.247,0.247,01,01003010400
3,5321.0,41.5,0.255,0.842,0.129,0.179,41944.0,131100.0,1975.0,0.929,0.321,0.147,0.010,0.172,01,01003010500
4,3398.0,29.3,0.431,0.375,0.625,0.220,27587.0,92300.0,1978.0,0.981,0.434,0.185,0.199,0.410,01,01003010600


In [280]:
## reset index?
cleaned.to_pickle(f"{data_directory}/qozs_features.pkl")
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8685 entries, 0 to 8763
Data columns (total 16 columns):
population_total           8685 non-null float64
age_median                 8682 non-null float64
p_never_married            8685 non-null float64
p_white                    8685 non-null float64
p_black                    8685 non-null float64
p_poverty                  8685 non-null float64
household_income_median    8657 non-null float64
home_value_median          8382 non-null float64
structure_year_median      8661 non-null float64
p_outofcountyflux          7822 non-null float64
p_pop_renting              8679 non-null float64
p_units_vacant             8679 non-null float64
p_mobilehomes              8679 non-null float64
p_multiple_unit_strucs     8679 non-null float64
state                      8685 non-null object
tract                      8685 non-null object
dtypes: float64(14), object(2)
memory usage: 1.1+ MB
